<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using Apache Spark GraphFrames with SingleStore Notebooks</h1>
    </div>
</div>

In [4]:
!pip cache purge --quiet

In [5]:
!conda install -y --quiet -c conda-forge openjdk=8

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk=8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.12.14 |       hbcca054_0         153 KB  conda-forge
    certifi-2024.12.14         |     pyhd8ed1ab_0         158 KB  conda-forge
    libgcc-14.2.0              |       h77fa898_1         829 KB  conda-forge
    libgcc-ng-14.2.0           |       h69a702a_1          53 KB  conda-forge
    libgomp-14.2.0             |       h77fa898_1         450 KB  conda-forge
    openjdk-8.0.412            |       hd590300_1        88.3 MB  conda-forge
    openssl-3.4.0              |       hb9d3cd8_0         2.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        92.7

In [6]:
!pip install folium --quiet
!pip install graphframes --quiet
!pip install pyspark --quiet

In [8]:
import folium
import pandas as pd

from graphframes import GraphFrame
from pyspark.sql import SparkSession
from singlestoredb import notebook as nb
from singlestoredb.management import get_secret

In [9]:
# List of Maven coordinates for all required packages
maven_packages = [
    "graphframes:graphframes:0.8.4-spark3.5-s_2.12",
    "org.scala-lang:scala-library:2.12",
    "com.singlestore:singlestore-jdbc-client:1.2.4",
    "com.singlestore:singlestore-spark-connector_2.12:4.1.8-spark-3.5.0",
    "org.apache.commons:commons-dbcp2:2.12.0",
    "org.apache.commons:commons-pool2:2.12.0",
    "io.spray:spray-json_3:1.3.6"
]

# Create Spark session with all required packages
spark = (SparkSession
             .builder
             .config("spark.jars.packages", ",".join(maven_packages))
             .appName("Spark GraphFrames Test")
             .getOrCreate()
        )

spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
graphframes#graphframes added as a dependency
org.scala-lang#scala-library added as a dependency
com.singlestore#singlestore-jdbc-client added as a dependency
com.singlestore#singlestore-spark-connector_2.12 added as a dependency
org.apache.commons#commons-dbcp2 added as a dependency
org.apache.commons#commons-pool2 added as a dependency
io.spray#spray-json_3 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3887370f-45c8-4377-8dc5-45516a49ca5d;1.0
	confs: [default]
	found graphframes#graphframes;0.8.4-spark3.5-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
	found com.singlestore#singlestore-jdbc-client;1.2.4 in central
	found com.singlestore#singlestore-spark-connector_2.12;4.1.8-spark-3.5.0 in central
	found org.apache.avro#avro;1.11.3 in central
	found com.fasterxml.jackson.core#jackson-core;2.14.2 in central
	fo

In [11]:
%%sql
DROP DATABASE IF EXISTS spark_demo;
CREATE DATABASE IF NOT EXISTS spark_demo;

1 rows affected.

1 rows affected.

++
||
++
++

In [13]:
%%sql
USE spark_demo;

DROP TABLE IF EXISTS connections;
CREATE ROWSTORE TABLE IF NOT EXISTS connections (
     src      INT,
     dst      INT,
     line     VARCHAR(32),
     colour   VARCHAR(8),
     time     INT,
     PRIMARY KEY(src, dst, line)
);

DROP TABLE IF EXISTS stations;
CREATE ROWSTORE TABLE IF NOT EXISTS stations (
     id          INT PRIMARY KEY,
     latitude    DOUBLE,
     longitude   DOUBLE,
     name        VARCHAR(32),
     zone        FLOAT,
     total_lines INT,
     rail        INT
);

++
||
++
++

In [14]:
connections_url = "https://raw.githubusercontent.com/VeryFatBoy/singlestore-geospatial-example/refs/heads/main/datasets/london_connections.csv"
stations_url = "https://raw.githubusercontent.com/VeryFatBoy/singlestore-geospatial-example/refs/heads/main/datasets/london_stations.csv"
lines_url = "https://raw.githubusercontent.com/VeryFatBoy/singlestore-geospatial-example/refs/heads/main/datasets/london_lines.csv"

connections_df = pd.read_csv(connections_url)
connections_df.rename(
    columns = {"station1": "src", "station2": "dst"},
    inplace = True
)

stations_df = pd.read_csv(stations_url)
stations_df.drop(
    "display_name",
    axis = 1,
    inplace = True
)

lines_df = pd.read_csv(lines_url)
lines_df.drop(
    "stripe",
    axis = 1,
    inplace = True
)

connections_df = pd.merge(
    connections_df,
    lines_df,
    on = "line",
    how = "left"
)
connections_df.drop(
    "line",
    axis = 1,
    inplace = True
)
connections_df.rename(
    columns = {"name": "line"},
    inplace = True
)

In [15]:
London = [51.509865, -0.118092]
mymap = folium.Map(location = London, zoom_start = 12)

# Add markers for stations
for idx, row in stations_df.iterrows():
    folium.Marker(
        [row["latitude"], row["longitude"]],
        popup = row["name"]
    ).add_to(mymap)

# Add lines with colours
for idx, row in connections_df.iterrows():
    source = stations_df.loc[stations_df["id"] == row["src"]]
    target = stations_df.loc[stations_df["id"] == row["dst"]]
    
    # Extract latitude and longitude
    source_coords = (float(source["latitude"].iloc[0]), float(source["longitude"].iloc[0]))
    target_coords = (float(target["latitude"].iloc[0]), float(target["longitude"].iloc[0]))
    
    folium.PolyLine(
        locations = [source_coords, target_coords],
        color = row["colour"]
    ).add_to(mymap)

html_content = mymap._repr_html_()

In [16]:
with nb.stage.open("map.html", "w") as st:
    st.write(html_content)

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [17]:
from sqlalchemy import *

db_connection = create_engine(connection_url)
url = db_connection.url

In [18]:
connections_df.to_sql(
    "connections",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

406

In [19]:
stations_df.to_sql(
    "stations",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

302

In [20]:
%%sql
SELECT * FROM connections LIMIT 5;

5 rows affected.

src,dst,line,colour,time
1,73,Piccadilly Line,#003688,2
7,145,Northern Line,#000000,2
8,264,Northern Line,#000000,2
11,163,Bakerloo Line,#B36305,1
16,173,Central Line,#E32017,2


In [21]:
%%sql
SELECT * FROM stations LIMIT 5;

5 rows affected.

id,latitude,longitude,name,zone,total_lines,rail
21,51.5403,0.127,Becontree,5.0,1,0
25,51.512,-0.1031,Blackfriars,1.0,2,0
31,51.6071,-0.1243,Bounds Green,3.5,1,0
39,51.5481,-0.1188,Caledonian Road,2.0,1,0
42,51.5051,-0.0209,Canary Wharf,2.0,2,0


In [22]:
password = get_secret("password")
host = url.host
port = url.port
cluster = host + ":" + str(port)

In [23]:
spark.conf.set("spark.datasource.singlestore.ddlEndpoint", cluster)
spark.conf.set("spark.datasource.singlestore.user", "admin")
spark.conf.set("spark.datasource.singlestore.password", password)
spark.conf.set("spark.datasource.singlestore.disablePushdown", "false")

In [24]:
connections = (spark.read
    .format("singlestore")
    .load("spark_demo.connections")
)

In [25]:
stations = (spark.read
    .format("singlestore")
    .load("spark_demo.stations")
)

In [26]:
underground = GraphFrame(stations, connections)

/opt/conda/lib/python3.11/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [27]:
underground.vertices.show(5)

+---+--------+---------+----------------+----+-----------+----+
| id|latitude|longitude|            name|zone|total_lines|rail|
+---+--------+---------+----------------+----+-----------+----+
| 48| 51.5185|  -0.1111|   Chancery Lane| 1.0|          1|   0|
| 83| 51.5203|    -0.17|Edgware Road (C)| 1.0|          3|   0|
|121| 51.6503|  -0.1943|     High Barnet| 5.0|          1|   0|
|203| 51.5093|   0.0336|   Prince Regent| 3.0|          1|   0|
|283|  51.521|  -0.2011| Westbourne Park| 2.0|          1|   0|
+---+--------+---------+----------------+----+-----------+----+
only showing top 5 rows



In [28]:
underground.edges.show(5)

+---+---+--------------------+-------+----+
|src|dst|                line| colour|time|
+---+---+--------------------+-------+----+
| 20| 65|Docklands Light R...|#00A4A7|   2|
| 66| 85|       District Line|#00782A|   3|
| 92|145|Hammersmith & Cit...|#F3A9BB|   4|
|127|226|        Central Line|#E32017|   1|
|185|281|       Bakerloo Line|#B36305|   2|
+---+---+--------------------+-------+----+
only showing top 5 rows



In [29]:
(underground
    .vertices
    .groupBy("zone")
    .count()
    .orderBy("count", ascending = False)
    .show()
)

+----+-----+
|zone|count|
+----+-----+
| 2.0|   75|
| 1.0|   60|
| 3.0|   47|
| 4.0|   38|
| 5.0|   28|
| 6.0|   18|
| 2.5|   17|
| 3.5|    6|
| 1.5|    4|
| 8.0|    2|
|10.0|    2|
| 7.0|    2|
| 9.0|    1|
| 6.5|    1|
| 5.5|    1|
+----+-----+



In [30]:
(underground
    .edges
    .filter("line = 'District Line'")
    .count()
)

59

In [31]:
(underground
    .vertices
    .groupBy()
    .max("total_lines")
    .show()
)

+----------------+
|max(total_lines)|
+----------------+
|               6|
+----------------+



In [32]:
(underground
    .vertices
    .filter("total_lines == 6")
    .show()
)

+---+--------+---------+--------------------+----+-----------+----+
| id|latitude|longitude|                name|zone|total_lines|rail|
+---+--------+---------+--------------------+----+-----------+----+
|145| 51.5308|  -0.1238|King's Cross St. ...| 1.0|          6|   1|
+---+--------+---------+--------------------+----+-----------+----+



In [33]:
spark.stop()

## Cleanup

In [34]:
%%sql
DROP TABLE IF EXISTS connections;
DROP TABLE IF EXISTS stations;

++
||
++
++

In [35]:
%%sql
DROP DATABASE IF EXISTS spark_demo;

++
||
++
++